In [5]:
import pandas as pd
import numpy as np
import requests
from pathlib import Path
import time
import time
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select

In [6]:
import requests
import zipfile
import io
import re
import subprocess
from sys import platform


class GetChromedriver:
    LATEST_URL = "http://chromedriver.storage.googleapis.com/LATEST_RELEASE"
    BASE_URL = "http://chromedriver.storage.googleapis.com/"
    MAC_FILENAME = "chromedriver_mac64.zip"
    LINUX_FILENAME = "chromedriver_linux64.zip"
    TARGET_DIR = "C:/Users/manfye/Desktop/Own Stuffs/stroke_map"  # change me

    def __init__(self):
        self.latest = self._get_latest_version()
        self.installed = self._get_installed_version()
        self.download_url = self._build_download_url()

    def _get_latest_version(self):
        response = requests.get(self.LATEST_URL)
        text = response.text
        print(f"latest version is {text}")
        return text

    def _get_installed_version(self):
        try:
            installed = str(
                subprocess.run(["chromedriver", "-v"], stdout=subprocess.PIPE).stdout
            )
        except:
            installed = (
                "0.0.0.0"
            )  # this is the pattern of the versioning chromedriver is following.
        version = re.findall("\d+\.\d+\.\d+\.\d+", installed)[0]
        print(f"installed version is {installed}")
        print(f"extracted version number is {version}")
        return version

    def _build_download_url(self):
        filename = "chromedriver_win32.zip"
        url = f"{self.BASE_URL}{self.latest}/{filename}"
        print(f"download url is {url}")
        return url

    def download_extract(self):
        if self.latest != self.installed:
            print("downloading latest chromedriver")
            response = requests.get(self.download_url, stream=True)
            zip_file = zipfile.ZipFile(io.BytesIO(response.content))
            zip_file.extractall(self.TARGET_DIR)
            subprocess.run(["chmod", "0777", "C:/Users/manfye/Desktop/Own Stuffs/stroke_map/chromedriver.zip"])
        else:
            print("latest version is installed")


get_chromedriver = GetChromedriver()
get_chromedriver.download_extract()

latest version is 114.0.5735.90
installed version is b'ChromeDriver 114.0.5735.90 (386bc09e8f4f2e025eddae123f36f6263096ae49-refs/branch-heads/5735@{#1052})\r\n'
extracted version number is 114.0.5735.90
download url is http://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_win32.zip
latest version is installed


In [7]:
browser = webdriver.Chrome()
browser.get('https://mystrokehospital.my/hospital/')

df_full = pd.DataFrame()

In [29]:
from bs4 import BeautifulSoup
import pandas as pd

page_source = browser.page_source

# replace 'page_source' with the actual source string you get from Selenium
soup = BeautifulSoup(page_source, 'html.parser') 

data = {
    'Hospital': [],
    'Address': [],
    'Telephone': [],
    'Type': [],
}

for div in soup.select('div.hosp-list.col-2'):
    hospital = div.select_one('h2.hospital-title')
    address = div.select_one('span.map-address.result')
    tel = div.select_one('a.map-hosp-tel')
    type_ = div.select_one('span.map-type')

    if hospital:
        data['Hospital'].append(hospital.get_text(strip=True))
    else:
        data['Hospital'].append(None)
    
    if address:
        data['Address'].append(address.get_text(strip=True))
    else:
        data['Address'].append(None)
    
    if tel:
        data['Telephone'].append(tel.get_text(strip=True))
    else:
        data['Telephone'].append(None)
    
    if type_:
        data['Type'].append(type_.get_text(strip=True))
    else:
        data['Type'].append(None)

df = pd.DataFrame(data)

In [30]:
df_full = pd.concat([df_full, df])

In [31]:
df_full
# for review_selector in reviews_selector:
#     review_div = review_selector.find('div', class_='dyn_full_review')
#     if review_div is None:
#         review_div = review_selector.find('div', class_='basic_review')
#     review = review_div.find('div', class_='entry').find('p').get_text()
#     review = review.strip()
#     reviews.append(review)

,Hospital,Address,Telephone,Type
0,Ara Damansara Medical Centre,"Lot 2, Jalan Lapangan Terbang Subang, Seksyen ...",03-5639 1212,Private 24 Hours
1,Assunta Hospital,"Jalan Templer, Pjs 4, 46900 Petaling Jaya, Sel...",03-7872 3000,Private Normal Hours
2,AVISENA Specialist Hospital,"4, Jalan Ikhtisas, Seksyen 14, Shah Alam, Sela...",03-5515 1888,Private Normal Hours
3,Bagan Specialist Centre,"Jalan Bagan 1, Taman Bagan, 13400 Pulau Pinang",04-371 0000,Private 24 Hours
4,Borneo Medical Centre,"Lot 10992, 801-2B Jalan Tun Jugah, Kenny Hill,...",082-507 333,Private 24 Hours
...,...,...,...,...
7,Sunway Medical Centre (Comprehensive Stroke Ce...,"5, Jalan Lagoon Selatan, Bandar Sunway, 47500 ...",03-7491 9191,Private 24 Hours
8,Sunway Medical Centre Velocity,"Sunway Medical Centre Velocity, Lingkaran SV, ...",03-9772 9191 (Emergency Hotline),Private 24 Hours
9,Thomson Hospital Kota Damansara,"11, Jalan Teknologi, Pju 5 Kota Damansara, 478...",03-6287 1111,Private 24 Hours
0,Timberland Medical Centre,"5164-5165, Block 16 KLCD 2 1/2 Mile, Rock Road...",082-234466,Private Normal Hours


In [33]:
df_full.reset_index(drop=True).to_csv("my_stroke.csv")